# Citypy introduction 

In [ ]:
# You need to:
# 1) "pip install citipy" 
# 2) "pip install gmaps"
from citipy import citipy
import random
import requests
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import sys
import pandas as pd
import time
from scipy.stats import linregress
import gmaps
from weather_config import weather_api_key
from weather_config import g_key

In [ ]:
# Create a list of coordinates. See this list comprehension method
# for creating them. 
coordinates = [[random.uniform(-90, 90) ,random.uniform(-180, 180)] for x in range(3)]
coordinates

In [ ]:
#lats = np.random.uniform(-90,90, size=5)
#longs = np.random.uniform(-180,180, size=5)
#coordinates = zip(lats, longs)
#list(coordinates)

In [ ]:
# Loop through the coordinates and find the nearest city
for coord in coordinates:
    print(f"{citipy.nearest_city(coord[0], coord[1]).city_name},{citipy.nearest_city(coord[0], coord[1]).country_code}")

In [ ]:
#Module 6.2.2 explains how to get the api key and
# create config.py with the api key defined in it.
# I named mine weatherconfig.py.
# Remember to create a py file, not an ipynb file.
from weather_config import weather_api_key

In [ ]:
# Current weather URL for the cities
# See https://openweathermap.org/current for documentation
for coord in coordinates:
    city_name_code = f"{citipy.nearest_city(coord[0], coord[1]).city_name},{citipy.nearest_city(coord[0], coord[1]).country_code}"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name_code}&units=Imperial&appid={weather_api_key}"
    #print(url)
    result = requests.get(url)
    #print(result.json())
    if result.json().get("cod") == 200:
        weather_data = result.json()
        temperature = weather_data["main"]["temp"]
        
        # Get the datetime as a string, formatted to print
        temp_date_time = weather_data["dt"]
        weather_dt = dt.datetime.utcfromtimestamp(weather_data["dt"])
        weather_dt_string = weather_dt.strftime('date: %Y-%m-%d time: %H:%M:%S')
        
        print(f"Temperature in {city_name_code} is {temperature} degrees farenheit at {weather_dt_string}")
    else:
        print(f"Failed to get the weather for {city_name_code}")

# Now lets do this as in Module 6.2.6

In [ ]:
# Create a list of coordinates. See this list comprehension method for creating them. 
coordinates = [[random.uniform(-90, 90) ,random.uniform(-180, 180)] for x in range(5)]
coordinates

# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name 
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
#set_count = 1
#record_count = 1

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create a list to hold a dictionary for each city
city_data = []

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 5 for logging purposes.
    """
    if (i % 5 == 0 and i >= 5):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    #city_url = f"http://api.openweathermap.org/data/2.5/weather?q={city.replace(' ','+')}&units=Imperial&appid={weather_api_key}"
    print(city_url)

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    """
    print(f"Processing {city}")
    city_url = url + "&q=" + city.replace(" ","+")
    
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        print(city_weather)
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = dt.datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


In [ ]:
# Create a DataFrome from the city_data[]
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
# Reorder
new_column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']
#new_column_order
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

In [ ]:
# Build the scatter plot for latitude vs. max temperature.
plt.scatter(city_data_df["Lat"],
            city_data_df['Max Temp'],
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Max Temperature "+ time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Show plot.
plt.show()

In [ ]:
# Section 6.4.1 regression
(slope, intercept, r_value, p_value, std_err) = linregress(city_data_df["Lat"], city_data_df['Max Temp'])
# Get the equation of the line.
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
print(line_eq)
print(f"The p-value is: {p_value:.3f}")

In [ ]:
# Build the scatter plot for latitude vs. max temperature.
plt.scatter(city_data_df["Lat"],
            city_data_df['Max Temp'],
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")
#Plot the regression line
regress_values = [(lat * slope + intercept) for lat in city_data_df["Lat"]]
plt.plot(city_data_df["Lat"],regress_values,"r")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Max Temperature "+ time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Annotate the text for the line equation and add its coordinates.
plt.annotate(line_eq, (-57,10), fontsize=15, color="red")

# Show plot.
plt.show()

# Does latitude correlate to max temp?
Module 6.4.2

In [ ]:
# Northern Hemisphere
northern_df = city_data_df.loc[(city_data_df["Lat"] >= 0)]
plt.scatter(northern_df['Lat'],
            northern_df['Max Temp'],
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")
#Plot the regression line
(slope, intercept, r_value, p_value, std_err) = linregress(northern_df["Lat"], northern_df['Max Temp'])
regress_values = [(lat * slope + intercept) for lat in northern_df['Lat']]
plt.plot(northern_df['Lat'],regress_values,"r")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Max Temperature "+ time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Show plot.
plt.show()

In [ ]:
# Southern Hemisphere
southern_df = city_data_df.loc[(city_data_df["Lat"] < 0)]
plt.scatter(southern_df['Lat'],
            southern_df['Max Temp'],
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")
#Plot the regression line
(slope, intercept, r_value, p_value, std_err) = linregress(southern_df["Lat"], southern_df['Max Temp'])
regress_values = [(lat * slope + intercept) for lat in southern_df['Lat']]
plt.plot(southern_df['Lat'],regress_values,"r")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Max Temperature "+ time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Show plot.
plt.show()

# Gmaps!
Starting at Module 6.5.2

In [ ]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [ ]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


In [ ]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

# Getting hotel information

In [ ]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()
hotels

In [ ]:
for key in hotels.keys():
    print(key)

In [ ]:
for h in hotels['results']:
    print(f"{h['name']} : {h['geometry']['location']['lat']},{h['geometry']['location']['lng']}")

In [ ]:
# Lets draw hotel locations on a map!
# See: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
hotel_info = []
for h in hotels['results']:
    info = {'name': h['name'], 'location': (h['geometry']['location']['lat'], h['geometry']['location']['lng'])}
    hotel_info.append(info)
print(hotel_info)

In [ ]:
locations = [h['location'] for h in hotel_info]
marker_layer = gmaps.marker_layer(locations)
fig = gmaps.figure(center=(48.8566, 2.3522), zoom_level=12)
fig.add_layer(marker_layer)
fig

In [ ]:
#Same map but with info boxes
# https://docs.python.org/2.4/lib/node109.html
info_box_template = """
<dl>
<dt>Name: </dt><dd>{name}</dd>
</dl>
"""
info = [info_box_template.format(**h) for h in hotel_info]
marker_layer = gmaps.marker_layer(locations, info_box_content=info)
fig = gmaps.figure(center=(48.8566, 2.3522), zoom_level=12)
fig.add_layer(marker_layer)
fig